In [1]:
from numpy import log, exp, sqrt
from scipy import linalg
import numpy as np
from scipy.optimize import check_grad

In [2]:
# Prepare Data
from diabeticRetinopathyUtils import load_diabetic_retinopathy
X, y = load_diabetic_retinopathy(filename="diabeticRetinopathy.csv")


## Question1.1

### Computing the value, gradient and hessian of the objective function $f$

The goal is to find $ (w_0^*, w^*) = \underset{w_0 \in \mathbb{R}, w \in \mathbb{R}^n}{arg min} f(w_0, w)$ where $$f(w_0, w) = \frac{1}{n}\sum_{i=1}^n{log\left(1+ e^{-y_i\left(X_i^Tw+w_0\right)}\right)} + \frac{\rho}{2}{\|w\|}^2_2$$


The gradient will be :

$$ \nabla f(w_{0},w ) =\big ( \frac {\partial f}{\partial w_0} ,  \frac {\partial f}{\partial w} \big) = \big( \frac{1}{n}\sum_{i=1}^{n}\frac{-y_{i}}{1 + exp(y_{i}(X_{i}^T w + w_{0}))} , \frac{1}{n}\sum_{i=1}^{n}\frac{-X_{i} y_{i}}{1 + exp(y_{i}(X_{i}^T w + w_{0}))} + \rho w  \big)   $$


The Hessian matrix will be : 


Let $ z_{i} = X_{i}^T w +w_{0} $

$$ \frac {\partial^2 f}{\partial w_0 ^2} =  \frac {1}{n} \sum_{i=1}^{n} \frac {exp(y_{i}z_{i})y_{i}^2}{( 1 + exp(y_{i}z_{i}))^2}  $$

$$ \frac {\partial^2 f}{\partial w_0 \partial w} =  \frac {1}{n} \sum_{i=1}^{n} \frac {X_{i}^{T} y_{i}^2 exp(y_{i}z_{i})}{( 1 + exp(y_{i}z_{i}))^2} $$

$$ \frac {\partial^2 f}{\partial w \partial w_0} =  \frac {1}{n} \sum_{i=1}^{n} \frac {X_{i} y_{i}^2 exp(y_{i}z_{i})}{( 1 + exp(y_{i}z_{i}))^2} $$

$$ \frac {\partial^2 f}{\partial w ^2} =  \frac {1}{n} \sum_{i=1}^{n} \frac {X_{i}X_{i}^{T} y_{i}^2  exp(y_{i}z_{i})}{( 1 + exp(y_{i}z_{i}))^2}  + \rho I$$

$Hessian(f)$ =
\begin{pmatrix} 
\frac {\partial^2 f}{\partial w_0 ^2} &  \frac {\partial^2 f}{\partial w_0 \partial w}\\
\frac {\partial^2 f}{\partial w \partial w_0} & \frac {\partial^2 f}{\partial w ^2} 
\end{pmatrix}

We can see that every element of the hessian matrix is positive, so we can say that the hessian matrix is positive and the objetive funtion is convex.

## Question1.2

In [3]:
rho = 1.0 / X.shape[0]
w=np.ones(X.shape[1])


In [4]:
def f(w0, w, X, y, rho):
    """




    """
    n, m = X.shape
    z = [np.dot(X[i], w) + w0 for i in range(n)]

    # VALUE
    summands = [log(1 + exp(-y[i] * z[i])) for i in range(n)]
    func = (1 / n) * sum(summands) + (rho / 2) * np.linalg.norm(w)**2

    # GRADIENT
    summands = [-y[i] / (1 + exp(y[i] * z[i])) for i in range(n)]
    grad_w0 = (1 / n) * sum(summands)

    summands = [-X[i] * y[i] / (1 + exp(y[i] * z[i])) for i in range(n)]
    grad_w = (1 / n) * sum(summands) + rho * w

    # HESSIAN

    ex = [exp(y[i] * z[i]) for i in range(n)]

    block0 = [ex[i] * y[i]**2 / (1 + ex[i])**2 for i in range(n)]

    block1 = [X[i] * (ex[i] * y[i]**2 / (1 + ex[i])**2) for i in range(n)]

    block2 = [np.outer(X[i], X[i]) * ex[i] * y[i]**2 /
              (1 + ex[i])**2 for i in range(n)]

    block0 = (1 / n) * sum(block0)

    block1 = (1 / n) * sum(block1)

    block2 = (1 / n) * sum(block2) + rho * np.eye(m)

    hessian = np.block([
        [block0,      block1.reshape(1, m)],
        [block1.reshape(m, 1),  block2]
    ])

    return func, np.append(grad_w0, grad_w), hessian

In [5]:
def func(w_):
    w0 = w_[0]
    w = w_[1:]
    val, grad, hessian = f(w0, w, X, y, rho)
    return val


def grad(w_):
    w0 = w_[0]
    w = w_[1:]
    val, grad, hessian = f(w0, w, X, y, rho)
    return grad

In [6]:
rho = 1.0 / X.shape[0]
epsilon = 1e-10
w0 = 0.0
w = np.zeros(X.shape[1])

print(check_grad(func, grad, np.append(w0, w)))

5.56968199646e-06


## Question1.3

In [7]:
def Newton(f, w0, w, X, y, rho, epsilon):

    val, grad, hessian = f(w0, w, X, y, rho)

    norm = np.linalg.norm(grad[1:])
    k = 0

    while(norm > epsilon):
        l = np.append(w0, w) - np.linalg.solve(hessian, grad)
        w0 = l[0]
        w = l[1:]

        val, grad, hessian = f(w0, w, X, y, rho)

        norm = np.linalg.norm(grad[1:])

        k = k + 1

    return val, np.append(w0,w), k

In [8]:
rho = 1.0 / X.shape[0]
epsilon=10e-10
w0=0.0
w=np.zeros(X.shape[1])

val,w_,k=Newton(f, w0, w, X, y, rho, epsilon)

w_


array([ 0.67731198,  0.28526576, -0.24554742,  5.61365006,  0.9888679 ,
       -1.82791918, -2.55695256, -1.58747255,  0.43396361,  0.48515529,
       -0.40441764,  0.17280405, -0.57564945,  0.43832626, -0.62435831,
        1.25356177,  0.72621094, -0.00607139, -0.1490878 , -0.09505212])

## Question1.4

In [29]:
rho = 1.0 / X.shape[0]
epsilon=10e-10
w0=0.3
w=0.3*np.ones(X.shape[1])

val,w_,k=Newton(f, w0, w, X, y, rho, epsilon)

/Users/alexandrema/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in double_scalars
/Users/alexandrema/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: overflow encountered in double_scalars
/Users/alexandrema/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in double_scalars
/Users/alexandrema/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':
/Users/alexandrema/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()
/Users/alexandrema/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
/Users/alexandrema/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in exp
/Users/alexandrema/anacond

####  The Problem of overflow !!

## Question1.5

In [38]:
def func(w_):
    w0 = w_[0]
    w = w_[1:]
    val, grad, hessian = f(w0, w, X, y, rho)
    return val


def armijo(w_,func,grad,hessi_grad,a,b):
    
    """
    Search for the next step according Armijo condition
    
    Input:
   
    func= the objective function
    grad = the gradient of objective function
    hessian =
    a = 
    b = 
    
    Output:
    
    the value of step
   
    """
    k=1.0
    beta =0.5
    gamma =b*(a**k)
    
    
    while ( func(w_ - gamma*hessi_grad) > func(w_) -beta*gamma*np.sum(grad*hessi_grad) ):
        k=k+1.0
        gamma= b*(a**k)
    
            
    return gamma

In [48]:
def NewtonLinearSearch(f, w0, w, X, y, rho, epsilon,numMax):

    val, grad, hessian = f(w0, w, X, y, rho)

    norm = np.linalg.norm(grad[1:])
    k = 0.0 
    a = 0.5
    gamma = 1.0

    
    hessi_grad=np.linalg.solve(hessian, grad)
    
    while(norm > epsilon and k<numMax):
        
        gamma = armijo(np.append(w0, w),func,grad, hessi_grad,a,2*gamma)
        l = np.append(w0, w) - gamma*hessi_grad
        w0 = l[0]
        w = l[1:]

        val, grad, hessian = f(w0, w, X, y, rho)
        hessi_grad =np.linalg.solve(hessian, grad)
        
        norm = np.linalg.norm(grad[1:])
    
        k = k + 1

    return val, np.append(w0,w), k

In [49]:
rho = 1.0 / X.shape[0]
epsilon=10e-10
w0=0.3
w=0.3*np.ones(X.shape[1])

val,w_,k=NewtonLinearSearch(f, w0, w, X, y, rho, epsilon,30)

In [50]:
w_

array([ 0.67731193,  0.28526576, -0.24554742,  5.6136499 ,  0.98886785,
       -1.82791914, -2.55695248, -1.58747249,  0.4339636 ,  0.48515529,
       -0.40441763,  0.17280406, -0.57564946,  0.43832627, -0.62435828,
        1.25356167,  0.72621087, -0.00607139, -0.1490878 , -0.09505212])